In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
import pickle 
pd.set_option('display.max_columns', None)

In [2]:
# Load in our data
big_df = pd.read_csv("../data/big_df.csv")
big_df = big_df.drop(columns="lev1")
big_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (215,216) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,7.088018,-6.329031,17.385159,9.229631,10.297141,15.558662,1979-10-28,1983-02-24,0.672043,0.824176,0,0,0.664179,0.729730,0.259259,0.402516,0.565517,0.732824,0.367347,0.274725,0.378099,0.441397,0.606941,0.628480,0.725490,0.748148,0.201258,0.240000,0.467636,0.418054,0.60293

In [3]:
# removing fights that have not taken place yet
df = big_df[big_df["winner"] != "tbd"]
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
15,13.050734,-1.564451,17.343435,0.793243,4.292702,2.357694,1993-09-20,1991-04-21,0.148368,1.253918,0,0,0.811765,0.828025,0.422535,0.342857,0.613333,0.738095,0.370968,0.222222,0.396283,0.552301,0.652708,0.533101,0.725191,0.648649,0.428571,0.200000,0.392017,0.459318,0.73726

In [4]:
# Dropping all fights that occured before unified rules were implamented 
df["date"] = pd.to_datetime(df["date"])
df = df[df["date"] >= "November 17, 2000"]
df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
15,13.050734,-1.564451,17.343435,0.793243,4.292702,2.357694,1993-09-20,1991-04-21,0.148368,1.253918,0,0,0.811765,0.828025,0.422535,0.342857,0.613333,0.738095,0.370968,0.222222,0.396283,0.552301,0.652708,0.533101,0.725191,0.648649,0.428571,0.200000,0.392017,0.459318,0.73726

In [5]:
df["red_age"] = pd.Series(df["red_age"], dtype="int")
df["blue_age"] = pd.Series(df["blue_age"], dtype="int")
df["red_heightIN"] = pd.Series(df["red_heightIN"], dtype="int")
df["blue_heightIN"] = pd.Series(df["blue_heightIN"], dtype="int")
df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
15,13.050734,-1.564451,17.343435,0.793243,4.292702,2.357694,1993-09-20,1991-04-21,0.148368,1.253918,0,0,0.811765,0.828025,0.422535,0.342857,0.613333,0.738095,0.370968,0.222222,0.396283,0.552301,0.652708,0.533101,0.725191,0.648649,0.428571,0.200000,0.392017,0.459318,0.73726

In [6]:
df = df[df["red_age"] < 75]
df = df[df["blue_age"] < 75]
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
15,13.050734,-1.564451,17.343435,0.793243,4.292702,2.357694,1993-09-20,1991-04-21,0.148368,1.253918,0,0,0.811765,0.828025,0.422535,0.342857,0.613333,0.738095,0.370968,0.222222,0.396283,0.552301,0.652708,0.533101,0.725191,0.648649,0.428571,0.200000,0.392017,0.459318,0.73726

In [7]:
df = df[df["red_heightIN"] < 100]
df = df[df["blue_heightIN"] < 100]
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
15,13.050734,-1.564451,17.343435,0.793243,4.292702,2.357694,1993-09-20,1991-04-21,0.148368,1.253918,0,0,0.811765,0.828025,0.422535,0.342857,0.613333,0.738095,0.370968,0.222222,0.396283,0.552301,0.652708,0.533101,0.725191,0.648649,0.428571,0.200000,0.392017,0.459318,0.73726

In [8]:
# Label encoder for stance and weight class
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
df['red_stance'] = number.fit_transform(df['red_stance'].astype('str'))
df['blue_stance'] = number.fit_transform(df['blue_stance'].astype('str'))
df['weight_class'] = number.fit_transform(df['weight_class'].astype('str'))
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
15,13.050734,-1.564451,17.343435,0.793243,4.292702,2.357694,1993-09-20,1991-04-21,0.148368,1.253918,0,0,0.811765,0.828025,0.422535,0.342857,0.613333,0.738095,0.370968,0.222222,0.396283,0.552301,0.652708,0.533101,0.725191,0.648649,0.428571,0.200000,0.392017,0.459318,0.73726

In [9]:
# removing draws and no contests 
df = df[df["winner"] != "Draw"] # it's arguable draws are useful datapoints, but I choose to remove them
df = df[df["winner"] != "NC"] # no contests however are not useful datapoints
df["winner"].value_counts()

red     3869
blue    2046
Name: winner, dtype: int64

In [10]:
df = df[df["method"] != "DQ"]
df["method"].value_counts()

UD            2161
KO/TKO        1965
Submission    1151
SD             577
MD              43
Name: method, dtype: int64

In [11]:
df

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
15,13.050734,-1.564451,17.343435,0.793243,4.292702,2.357694,1993-09-20,1991-04-21,0.148368,1.253918,0,0,0.811765,0.828025,0.422535,0.342857,0.613333,0.738095,0.370968,0.222222,0.396283,0.552301,0.652708,0.533101,0.725191,0.648649,0.428571,0.200000,0.392017,0.459318,0.73726

In [12]:
# setting our X and y values
X = df.drop(columns=["winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method", "location", \
                    "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"])
y = df["winner"]
feature_names = X.columns

In [13]:
# illustrate the concept of resampling with a logistic regression model before applying it to our random forest
log_0 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_0 = log_0.predict(X)
pred_y_0

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['blue', 'red', 'red', ..., 'red', 'red', 'red'], dtype=object)

In [14]:
# How's the accuracy?
print(accuracy_score(pred_y_0, y))

0.6652535187383415


In [15]:
print( np.unique( pred_y_0 ) )

['blue' 'red']


In [16]:
df["winner"].value_counts()

red     3857
blue    2040
Name: winner, dtype: int64

In [17]:
# Separate majority and minority classes
df_majority = df[df.winner=="red"]
df_minority = df[df.winner=="blue"]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority), # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.winner.value_counts()

red     3857
blue    3857
Name: winner, dtype: int64

In [18]:
df_upsampled

,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
17,-27.700000,6.684600,2.366667,15.790182,30.066667,9.105582,1995-09-29,1987-11-28,0.000000,0.000000,0,0,0.529412,0.658824,0.303030,0.444444,0.800000,0.814815,0.272727,0.333333,0.492823,0.448529,0.524390,0.630769,0.833333,0.789474,0.222222,0.153846,0.468085,0.442308,0.5410

In [19]:
# Separate input features (X) and target variable (y)
y = df_upsampled.winner
X = df_upsampled.drop(columns=["winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method", \
                               "location", "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"], axis=1)
 

feature_names = X.columns



# Train model
log_1 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_1 = log_1.predict(X)
 
# Is our model still predicting just one class?
print( np.unique( pred_y_1 ) )

['blue' 'red']


C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
# How's our accuracy?
print(accuracy_score(y, pred_y_1))

0.6061705989110708


In [21]:
# Now that we can see resampling will help us increase accuracy on a logistic regression, lets see on random forest
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [22]:
# our decision tree score has increased dramatically
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7765681700362882

In [23]:
sorted(zip(clf.feature_importances_, X), reverse=True)

[(0.033266456776169635, 'red_StkDistAbsPerMin'),
 (0.027728807809083093, 'red_age'),
 (0.023748130869324924, 'blue_age'),
 (0.022579011042730145, 'blue_reachIN'),
 (0.021588146859654177, 'red_SigDistStkDef'),
 (0.015978672712546646, 'blue_SigBodyStkDef'),
 (0.015283522671979181, 'blue_TDsAbsPerMin'),
 (0.01497328259416213, 'red_TotStkAcc'),
 (0.014665201360897087, 'red_SigGndStkAcc'),
 (0.013980284624719924, 'red_SigStkPerMinDiff'),
 (0.013470574645070415, 'blue_StkClchAbsPerMin'),
 (0.01344576119798439, 'red_reachIN'),
 (0.013426047880549238, 'red_winPCT'),
 (0.013011616538162563, 'red_SigClchStkAcc'),
 (0.012309265375593747, 'red_SigBodyStkDef'),
 (0.012213174282809311, 'blue_heightIN'),
 (0.01216857431182761, 'blue_StkBodyAbsPerMin'),
 (0.01201129490639121, 'blue_CtrlDiff'),
 (0.011923855991561535, 'weight_class'),
 (0.011813193833022323, 'red_TDAcc'),
 (0.011452943987309477, 'red_StkClchAbsPerMin'),
 (0.011430178286797406, 'blue_sumSigDistSeen'),
 (0.011416727431988176, 'red_sumCtr

In [24]:
# and so has our random forest
rf = RandomForestClassifier(n_estimators=105)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8387765681700363

In [25]:
preds = rf.predict(X)

In [26]:
filename = "../saved_models/rf_up_model.sav"
pickle.dump(rf, open(filename, 'wb'))

In [27]:
# Repeating proccess by down sampling red wins

In [28]:
# Separate majority and minority classes
df_majority = df[df.winner=="red"]
df_minority = df[df.winner=="blue"]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority), # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.winner.value_counts()

red     2040
blue    2040
Name: winner, dtype: int64

In [29]:
# illustrating this concept with a logistic regression model before applying it to our random forest

In [30]:
# Separate input features (X) and target variable (y)
y = df_downsampled.winner
X = df_downsampled.drop(columns=["winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method", \
                               "location", "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"], axis=1)
 
# Train model
log_2 = LogisticRegression().fit(X, y)
 
# Predict on training set
pred_y_2 = log_2.predict(X)
 
# Is our model still predicting just one class?
pred_y_2

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array(['blue', 'red', 'blue', ..., 'red', 'red', 'blue'], dtype=object)

In [31]:
# How's our accuracy?
print( accuracy_score(y, pred_y_2) )

0.5997549019607843


In [32]:
# repeating on down-sampled data

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [34]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5441176470588235

In [35]:
sorted(zip(clf.feature_importances_, X), reverse=True)

[(0.04330891191636973, 'red_SigStkAbsPerMin'),
 (0.030693407488391028, 'red_age'),
 (0.029032351510127802, 'blue_age'),
 (0.019645940173597415, 'red_heightIN'),
 (0.019468996397844225, 'weight_class'),
 (0.019318903405634212, 'blue_reachIN'),
 (0.017794366240377866, 'red_CtrlDiff'),
 (0.016560038060683006, 'red_reachIN'),
 (0.015807961514095137, 'blue_TotStkAcc'),
 (0.014746889651355502, 'red_SoS'),
 (0.014565857707656865, 'red_TotStkAcc'),
 (0.014522749310357089, 'red_SigBodyStkDef'),
 (0.014198462293728379, 'red_SigGndStkDef'),
 (0.014088618291288359, 'red_TotStkDef'),
 (0.013716420047167176, 'red_CtrlSecPerMin'),
 (0.013458775818887481, 'blue_TDAcc'),
 (0.013155743008577171, 'red_SigClchStkDef'),
 (0.012719999421391827, 'red_StkClchPerMin'),
 (0.011983075835742235, 'blue_StkGndAbsPerMin'),
 (0.011929637835282997, 'red_CtrldSecPerMin'),
 (0.01106472599786463, 'red_TDsPerMin'),
 (0.010924730003032117, 'blue_SigBodyStkDef'),
 (0.010700224179226512, 'red_TotStkAbsPerMin'),
 (0.010611666

In [36]:
rf = RandomForestClassifier(n_estimators=105)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6049019607843137

In [37]:
preds = rf.predict(X)

In [38]:
sorted(zip(rf.feature_importances_, X), reverse=True)

[(0.016188106271678604, 'red_age'),
 (0.013763250972774276, 'blue_age'),
 (0.013350025381612287, 'red_reachIN'),
 (0.013117022712275434, 'blue_reachIN'),
 (0.012657774986116197, 'red_SigStkAbsPerMin'),
 (0.012258658180046877, 'red_heightIN'),
 (0.011959620539262451, 'blue_heightIN'),
 (0.010715797627126454, 'weight_class'),
 (0.01035400084957908, 'red_StkDistAbsPerMin'),
 (0.010161389579218033, 'red_StkHeadAbsPerMin'),
 (0.00944064704426619, 'red_TotStkPerMinDiff'),
 (0.009289033527618843, 'red_CtrlDiff'),
 (0.008727766917120475, 'red_TotStkAbsPerMin'),
 (0.007945949179170762, 'red_SigStkPerMinDiff'),
 (0.007635461904911939, 'red_TDsPerMin'),
 (0.007543502178656426, 'red_TotStkPerMin'),
 (0.00747750031156132, 'red_StkLegAbsPerMin'),
 (0.007457125578835345, 'red_SigDistStkDef'),
 (0.007452135824711553, 'red_StkBodyAbsPerMin'),
 (0.007449531668871619, 'blue_TotStkAcc'),
 (0.007286567245048192, 'red_SigHeadStkDef'),
 (0.007195384371021876, 'red_SigStkDef'),
 (0.007034681177822212, 'blue_T

In [39]:
# saving this model too, even though it preforms worse than our upsampled model to compare on future observations
filename = "../saved_models/rf_down_model.sav"
pickle.dump(rf, open(filename, "wb"))